In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [11]:
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.models import Model, Sequential
from keras.utils import plot_model, print_summary
from keras.layers.merge import concatenate

In [18]:
def inception_mod(in_layer, f1,f2_in, f2_out, f3_in, f3_out, f4_out):
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(in_layer)
    
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(in_layer)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(in_layer)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
        
    
    mp = MaxPooling2D((3,3),strides = (1,1), padding='same')(in_layer)
    mp= Conv2D(f4_out, (5,5), padding='same', activation='relu')(mp)
    
    out_layer = concatenate([conv1, conv3, conv5, mp], axis = -1)
    return out_layer

In [19]:
visible = Input(shape = (256,256,3))

In [20]:
layer = inception_mod(visible, 64,96,128,64,32,32)
layer = inception_mod(layer, 128,128,256,128,64,64)
model = Model(inputs = visible, outputs = layer)

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 256, 256, 96) 384         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 256, 256, 64) 256         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_11 

In [23]:
plot_model(model)